# Download GDEL

In [1]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits
file_list = [x for x in link_list if str.isdigit(x[0:4])] # mengambil file x yang string are digits 4 baris pertama
file_list2 = file_list[0:5]   # mengambil file di list GDELT yang 2 baris pertama
print(file_list2)

infilecounter = 0
outfilecounter = 0

import os.path
import urllib
import zipfile
import glob
import operator

local_path = 'D:/GDELT/'

fips_country_code = 'ID'

for compressed_file in file_list2[infilecounter:]:
    print(compressed_file)

    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path + compressed_file):
        print('downloading, '),
        urllib.request.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)
        print('done')

    # extract the contents of the compressed file to a temporary directory
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path + compressed_file, mode='r')
    z.extractall(path=local_path + 'tmp/')
    print('done')

    # parse each of the csv files in the working directory,
    print('parsing,'),
    for infile_name in glob.glob(local_path + 'tmp/*'):
#         outfile_name = local_path + 'country/' + fips_country_code + str.isdigit(x[0:4] + '%04i.tsv' % outfilecounter
        outfile_name = local_path + 'country/' + fips_country_code + compressed_file +'%05i.tsv' % outfilecounter
        # open the infile and outfile
        with open(infile_name, encoding='utf8', mode='r') as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):
                    outfile.write(line)
            outfilecounter += 1

        # delete the temporary file
        os.remove(infile_name)
    infilecounter += 1
    print('done')

['20181101.export.CSV.zip', '20181031.export.CSV.zip', '20181030.export.CSV.zip', '20181029.export.CSV.zip', '20181028.export.CSV.zip']
20181101.export.CSV.zip
downloading, 
done
extracting,
done
parsing,
done
20181031.export.CSV.zip
downloading, 
done
extracting,
done
parsing,
done
20181030.export.CSV.zip
downloading, 
done
extracting,
done
parsing,
done
20181029.export.CSV.zip
downloading, 
done
extracting,
done
parsing,
done
20181028.export.CSV.zip
downloading, 
done
extracting,
done
parsing,
done


# PICKING  DATA

In [4]:
import glob
import pandas as pd

local_path = 'D:/GDELT/'
fips_country_code = 'ID'
# Get the GDELT field names from a helper file
colnames = pd.read_excel('CSV.header.fieldids.xlsx', sheetname='Sheet1',
                         index_col='Column ID', usecols=1)['Field Name']
# print (colnames)
# Build DataFrames from each of the intermediary files
files = glob.glob(local_path + 'country/' + fips_country_code + '*')
DFlist = []
for active_file in files:
    print (files)
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID']))
    break

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path + 'backupsmall' + fips_country_code + '.pickle')

# once everythin is safely stored away, remove the temporary files
# for active_file in files:
#     os.remove(active_file)

['D:/GDELT/country\\ID20181028.export.CSV.zip00004.tsv', 'D:/GDELT/country\\ID20181029.export.CSV.zip00003.tsv', 'D:/GDELT/country\\ID20181030.export.CSV.zip00002.tsv', 'D:/GDELT/country\\ID20181031.export.CSV.zip00001.tsv', 'D:/GDELT/country\\ID20181101.export.CSV.zip00000.tsv']


# List Data dalam Folder

In [5]:
from os import listdir
from os.path import isfile, join
folder_id = 'D:/GDELT/country/'
files = [f for f in listdir(folder_id) if isfile(join(folder_id, f))]
print(files)

['ID20181028.export.CSV.zip00004.tsv', 'ID20181029.export.CSV.zip00003.tsv', 'ID20181030.export.CSV.zip00002.tsv', 'ID20181031.export.CSV.zip00001.tsv', 'ID20181101.export.CSV.zip00000.tsv']


In [7]:
# Open first file for sanity check
df_awal = pd.read_csv(folder_id + files[2],sep="\t",encoding='latin-1') # melihat sampel isi data
df_awal.head()

,798469840,20181029,201810,2018,2018.8192,IGOEUREEC,EUROPEAN COMMISSION,EUR,EEC,Unnamed: 9,...,9179538.1,4.3,"Juanda, Jawa Timur, Indonesia.2",ID.2,ID08.2,-7.3855.2,112.764.2,9179538.2,20181030,https://www.goldcoastbulletin.com.au/the-13-minutes-before-tragedy-struck-how-the-lion-air-disaster-unfolded/news-story/28de53a23285acd177d9f5386b6731fb?nk=c468b881f39c524cdee34dbbe0916e22-1540856817
0,798469934,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,...,-2679652,4,"Jakarta, Jakarta Raya, Indonesia",ID,ID04,-6.174440,106.8290,-2679652,20181030,https://www.channelnewsasia.com/news/asia/lion...
1,798470075,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,...,ID,1,United States,US,US,39.828175,-98.5795,US,20181030,https://www.unmultimedia.org/avlibrary/asset/2...
2,798470108,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,...,-2690876,4,"Pangkal Pinang, Indonesia (general), Indonesia",ID,ID00,-2.133330,106.1330,-2690876,20181030,https://www.channelnewsasia.com/news/asia/all-...
3,798470109,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,...,-2679652,4,"Jakarta, Jakarta Raya, Indonesia",ID,ID04,-6.174440,106.8290,-2679652,20181030,https://www.goldcoastbulletin.com.au/the-13-mi...
4,798470110,20181030,201810,2018,2018.8219,NaN,NaN,NaN,NaN,NaN,...,-2690876,4,"Jakarta, Jakarta Raya, Indonesia",ID,ID04,-6.174440,106.8290,-2679652,20181030,https://www.goldcoastbulletin.com.au/the-13-mi...


# Simpan ke MySQL

Setetelah di List filenya, kemudian semua file yang ada di D:/GDELT/country/ dimasukkan ke MySQL dengan nama tabel per setiap file

In [8]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

host = 'localhost'
port = '3306'
username = 'root'
password = ''
database = 'gdelt_content'

# Create Connection to database
engine = create_engine('mysql+pymysql://'+username+':'+password+'@'+host+':'+port+'/'+database)
'''engine = create_engine('mysql+pymysql://root: @localhost:3306/gdelt_content_id')'''

def run(sql):
    df = pd.read_sql_query(sql, engine)
    return df

In [10]:
# Read header / column names
colnames = list(pd.read_excel('CSV.header.fieldids.xlsx', sheet_name='CSV.header.dailyupdates'))

for berkas in files:
    print('Extracting ' + berkas)
    df_satuan = pd.DataFrame()
    
    # Important: If your ID0000.tsv contains header, you can delete the header
    df_satuan = pd.read_csv(folder_id + berkas, sep="\t", names= colnames, encoding='latin-1')
    df_satuan.to_sql(name = berkas, con = engine, if_exists = 'append', index = False) # Memasukkan data ke tabel sesuai dengan file

Extracting ID20181028.export.CSV.zip00004.tsv
Extracting ID20181029.export.CSV.zip00003.tsv
Extracting ID20181030.export.CSV.zip00002.tsv
Extracting ID20181031.export.CSV.zip00001.tsv
Extracting ID20181101.export.CSV.zip00000.tsv
